<a href="https://colab.research.google.com/github/anuj0456/deep_learning_for_coders/blob/master/competitions/he_exhibit_art_cb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install catboost

     |████████████████████████████████| 65.7MB 79kB/s 


In [3]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import KFold
from lightgbm import LGBMRegressor
import matplotlib.pyplot as plt
from functools import partial
import seaborn as sns
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('ignore')

# EDA

In [4]:
pd.set_option('max_columns', 50)
pd.set_option('max_rows', 6500)
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [6]:
X_train = train.drop(['Customer Id', 'Artist Name', 'Scheduled Date', 'Delivery Date', 'Customer Location', 'Cost'], axis=1)

y_train = train['Cost']

In [7]:
X_test = test.drop(['Customer Id', 'Artist Name', 'Scheduled Date', 'Delivery Date', 'Customer Location'], axis=1)

In [8]:
cat = ['Material','International', 'Express Shipment', 'Installation Included', 'Transport', 'Fragile', 'Customer Information', 'Remote Location']
for feature in cat:
  if X_train[feature].isnull().sum() > 0:
    X_train[feature] = X_train[feature].fillna(X_train[feature].mode()[0])
  if X_test[feature].isnull().sum() > 0:
    X_test[feature] = X_test[feature].fillna(X_test[feature].mode()[0])

In [9]:
cont = ['Artist Reputation', 'Height', 'Width', 'Weight', 'Price Of Sculpture', 'Base Shipping Price']
for feature in cont:
  if X_train[feature].isnull().sum() > 0:
    X_train[feature] = X_train[feature].fillna(X_train[feature].median())
  if X_test[feature].isnull().sum() > 0:
    X_test[feature] = X_test[feature].fillna(X_test[feature].median())

In [10]:
train['Scheduled Date'] = pd.to_datetime(train['Scheduled Date'], errors='coerce')
train['Delivery Date'] = pd.to_datetime(train['Delivery Date'], errors='coerce')
X_train['SD_day'] = train['Scheduled Date'].dt.day
X_train['SD_month'] = train['Scheduled Date'].dt.month
# X_train['SD_year'] = train['Scheduled Date'].dt.year
X_train['SD_weekday'] = train['Scheduled Date'].dt.dayofweek
X_train['DD_day'] = train['Delivery Date'].dt.day
X_train['DD_month'] = train['Delivery Date'].dt.month
# X_train['DD_year'] = train['Delivery Date'].dt.year
X_train['DD_weekday'] = train['Delivery Date'].dt.dayofweek
X_train['Time GAP'] = (train['Scheduled Date'] - train['Delivery Date']).dt.days

In [11]:
test['Scheduled Date'] = pd.to_datetime(test['Scheduled Date'], errors='coerce')
test['Delivery Date'] = pd.to_datetime(test['Delivery Date'], errors='coerce')
X_test['SD_day'] = test['Scheduled Date'].dt.day
X_test['SD_month'] = test['Scheduled Date'].dt.month
# X_test['SD_year'] = test['Scheduled Date'].dt.year
X_test['SD_weekday'] = test['Scheduled Date'].dt.dayofweek
X_test['DD_day'] = test['Delivery Date'].dt.day
X_test['DD_month'] = test['Delivery Date'].dt.month
# X_test['DD_year'] = test['Delivery Date'].dt.year
X_test['DD_weekday'] = test['Delivery Date'].dt.dayofweek
X_test['Time GAP'] = (test['Scheduled Date'] - test['Delivery Date']).dt.days

In [12]:
# df[['City','Pin Code']] = df['Customer Location'].apply( lambda x: pd.Series(str(x).split(","))) 

In [13]:
# from pandas.api.types import is_object_dtype
# print(df.shape)
# for i, col in enumerate(df.columns):
#   if is_object_dtype(df[col]):
#     df[col] = df[col].astype(str)
#   if i == 0 or i == 1 or i == 2 or i == 5 or i == 5 or i == 6:
#     print(f"Col Name: {col}, Unique: {df[col].nunique()}, Mean: {df[col].mean():.2f}, Std: {df[col].std():.2f}, Nan: {df[col].isnull().sum()}, Type: {df[col].dtypes}")
#   else:
#     print(f"Col Name: {col}, Unique: {df[col].nunique()}, Nan: {df[col].isnull().sum()}, Type: {df[col].dtypes}")

In [14]:
le = LabelEncoder()
for feature in cat:
  print(feature)
  le.fit(X_train[feature])
  X_train[feature] = le.transform(X_train[feature])
  X_test[feature] = le.transform(X_test[feature])

Material
International
Express Shipment
Installation Included
Transport
Fragile
Customer Information
Remote Location


In [15]:
sc1 = StandardScaler()
X_train[["Weight","Price Of Sculpture"]] = sc1.fit_transform(X_train[["Weight","Price Of Sculpture"]])
X_test[["Weight","Price Of Sculpture"]] = sc1.transform(X_test[["Weight","Price Of Sculpture"]])

In [22]:
def log1p(vec):
    return np.log1p(abs(vec))

In [23]:
y_train = log1p(y_train)

In [24]:
cat_names=['Material','International', 'Express Shipment', 'Installation Included', 'Transport', 'Fragile', 'Customer Information', 'Remote Location']
cont_names=['Artist Reputation', 'Height', 'Width', 'Weight', 'Price Of Sculpture', 'Base Shipping Price']
cat_features=[4,7,8,9,10,11,12,13,14,15,16,17,18,19,20]

In [26]:
X_train.head()

,Artist Reputation,Height,Width,Weight,Material,Price Of Sculpture,Base Shipping Price,International,Express Shipment,Installation Included,Transport,Fragile,Customer Information,Remote Location,SD_day,SD_month,SD_weekday,DD_day,DD_month,DD_weekday,Time GAP
0,0.26,17.0,6.0,-0.141070,1,-0.133634,16.27,1,1,0,0,0,1,0,7,6,6,3,6,2,4
1,0.28,3.0,3.0,-0.142660,1,-0.134437,15.00,0,0,0,1,0,1,0,6,3,0,5,3,6,1
2,0.07,8.0,5.0,-0.142591,3,-0.134649,21.18,0,0,0,1,1,1,1,9,3,0,8,3,6,1
3,0.12,9.0,8.0,-0.141471,0,-0.134552,16.31,0,0,0,1,0,0,1,24,5,6,20,5,2,4
4,0.15,17.0,6.0,-0.142557,0,-0.134851,11.94,1,1,1,0,0,1,0,18,12,6,14,12,2,4


In [27]:
seed = 0
n_splits = 10
shuffle=True
iterations = 50000
early_stopping_rounds = 400
verbose_eval = 0
baseline_rounds = 10
cb_learning_rate = 0.06

In [28]:
split = KFold(n_splits=n_splits, shuffle=shuffle, random_state=seed)

In [40]:
cb_params = {'iterations':iterations,
             'learning_rate':cb_learning_rate,
             'depth':10,
             'bootstrap_type':'Poisson',
             'random_strength':1,
             'min_data_in_leaf':10,
             'l2_leaf_reg':3,
             'loss_function':'RMSE', 
             'eval_metric':'RMSE',
             'random_seed':seed,
             'grow_policy':'Depthwise',
             'max_bin':1024, 
             'model_size_reg': 0,
             'task_type': 'GPU',
             'od_type':'IncToDec',
             'od_wait':100,
             'metric_period':500,
             'verbose':verbose_eval,
             'subsample':0.6,
             'od_pval':1e-10,
             'max_ctr_complexity': 8,
             'has_time': False,
             'simple_ctr' : 'FeatureFreq',
             'combinations_ctr': 'FeatureFreq'
            }


In [ ]:
preds_list_cb = []
oof_cb = np.zeros((len(train)))
oof_cbx = np.zeros((len(train)))

for fold, (train_idx, val_idx) in enumerate(split.split(X_train)):
    print(f'Fold {fold+1}')
    X_tr = X_train.iloc[train_idx]
    X_val = X_train.iloc[val_idx]
    y_tr = y_train.iloc[train_idx]
    y_val = y_train.iloc[val_idx]
   
    ######CATBOOST#######
    ptrain = Pool(data=X_tr, label=y_tr, cat_features=cat_features)
    pvalid = Pool(data=X_val, label=y_val, cat_features=cat_features)
    ptest = Pool(data=X_test, cat_features=cat_features)
    CModel = CatBoostRegressor(**cb_params)
    CModel.fit(ptrain, eval_set=pvalid, use_best_model=True,early_stopping_rounds=early_stopping_rounds)
    temp_fold_preds = CModel.predict(pvalid)
    oof_cb[val_idx] = temp_fold_preds
    first_cb_rmse = mean_squared_error(y_val, temp_fold_preds, squared=False)
    print(f'RMSE of CB model is {first_cb_rmse}')
    baseline_preds_tr_cb = CModel.predict(ptrain)
    baseline_preds_vl_cb = CModel.predict(pvalid)
    test_preds_cb = CModel.predict(ptest)   

    
    for baseline in range(baseline_rounds):
        print(f'Using prediction of each model as a baseline for the next model round {baseline+1}')
        ptrain = Pool(data=X_tr, label=y_tr, cat_features=cat_features, baseline=baseline_train)
        pvalid = Pool(data=X_val, label=y_val, cat_features=cat_features, baseline=baseline_valid)
        ptest = Pool(data=X_test, cat_features=cat_features, baseline=baseline_test)
        cb_params_ = cb_params.copy()
        cb_params_.update({'learning_rate': cb_learning_rate*(1/(baseline+1))})
        CModel = CatBoostRegressor(**cb_params_)
        CModel.fit(ptrain, eval_set=pvalid, use_best_model=True, early_stopping_rounds=early_stopping_rounds)
        temp_fold_preds = CModel.predict(pvalid)
        oof_cbx[val_idx] = temp_fold_preds
        second_cb_rmse = mean_squared_error(y_val, temp_fold_preds, squared=False)
        print(f'RMSE of CB model with baseline round {baseline+1} {second_cb_rmse}')   
        baseline_train = CModel.predict(ptrain)
        baseline_valid = CModel.predict(pvalid)
        baseline_test = CModel.predict(ptest)

       
    print('-' * 100)
    print('',end='\n')
    preds_list_cb.append(CModel.predict(ptest))

In [ ]:
first_cb_rmse = mean_squared_error(y_train, oof_cb, squared=False)
second_cb_rmse = mean_squared_error(y_train, oof_cbx, squared=False)
print(f'RMSE for CB model is {first_cb_rmse}')
print(f'RMSE for CB model with XGB baseline is {second_cb_rmse}')
print(f'RMSE for CB and XGB blend is {mean_squared_error(y_train, (oof_cbx+oof_xgbx)/2, squared=False)}')

In [45]:
def expm1(x):
    return np.expm1(x)

In [46]:
def clipExp(vec):
    return np.clip(expm1(vec), 0, None)

In [47]:
preds1 = np.mean(preds_list_cb, axis=0)
preds1 = clipExp(preds1)
submission_cb = pd.DataFrame({'Customer Id':test['Customer Id'],'Cost':preds1})
submission_cb.to_csv('submission_cb.csv', index=False)
submission_cb.head()

,Customer Id,Cost
0,fffe3400310033003300,247.922378
1,fffe3600350035003400,259.818089
2,fffe3700360030003500,186.015421
3,fffe350038003600,208.129817
4,fffe3500390032003500,339.461715
